In [1]:
import numpy as np
import pandas as pd
from src.utils.utils import get_data, export_data

## Constants

In [15]:
processor_rows = set((642,647,649,659,667,701,750,759,819,859,883,884,919,927,929,932,1002))
ram_rows = set((441,485,534,553,584,610,613,642,647,649,659,667,701,750,759,819,859,884,919,927,929,932,990,1002))
battery_rows = set((113,151,309,365,378,441,450,553,584,610,613,630,642,647,649,659,667,701,750,756,759,764,819,855,859,884,915,916,927,929,932,990,1002))
display_rows = set((378,441,450,553,584,610,613,630,642,647,649,659,667,701,750,759,764,819,859,884,915,916,927,929,932,990,1002))
camera_rows = set((100,113,151,157,161,238,273,308,309,323,324,365,367,378,394,441,450,484,506,534,553,571,572,575,584,610,613,615,630,642,647,649,659,667,684,687,705,711,723,728,750,756,759,764,792,819,846,854,855,858,883,884,896,915,916,927,929,932,945,956,990,995,1002,1016 ))

In [2]:
df = get_data("smartphones", "raw")

In [3]:
df.columns

Index(['model', 'price', 'rating', 'sim', 'processor', 'ram', 'battery',
       'display', 'camera', 'card', 'os'],
      dtype='object')

In [4]:
df.head()

,model,price,rating,sim,processor,ram,battery,display,camera,card,os
0,OnePlus 11 5G,"₹54,999",89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,Memory Card Not Supported,Android v13
1,OnePlus Nord CE 2 Lite 5G,"₹19,989",81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 695, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
2,Samsung Galaxy A14 5G,"₹16,499",75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Exynos 1330, Octa Core, 2.4 GHz Processor","4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,"Memory Card Supported, upto 1 TB",Android v13
3,Motorola Moto G62 5G,"₹14,999",81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 695, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
4,Realme 10 Pro Plus,"₹24,999",82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Dimensity 1080, Octa Core, 2.6 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,Memory Card Not Supported,Android v13


## 1. Data Assessing

### 1.1 Quality Issues

1. **model** - some brands are written diiferently like OPPO in model column `consistency`
2. **price** - has unneccesary '₹' `validity`
3. **price** - has ',' between numbers `validity`
4. **price** - phone Namotel has a price of 99 `accuracy`
5. **ratings** - missing values `completeness`
6. **processor** - has some incorrect values for some samsung phones(row # -642,647,649,659,667,701,750,759,819,859,883,884,919,927,929,932,1002) `validity`
7. There is ipod on row 756 `validity`
8. **memory** - incorrect values in rows (441,485,534,553,584,610,613,642,647,649,659,667,701,750,759,819,859,884,919,927,929,932,990,1002) `validity`
9. **battery** - incorrect values in rows(113,151,309,365,378,441,450,553,584,610,613,630,642,647,649,659,667,701,750,756,759,764,819,855,859,884,915,916,927,929,932,990,1002) `validity`
10. **display** - sometimes frequency is not available `completeness`
11. **display** - incorrect values in rows(378,441,450,553,584,610,613,630,642,647,649,659,667,701,750,759,764,819,859,884,915,916,927,929,932,990,1002) `validity`
12. certain phones are foldable and the info is scattered `validity`
13. **camera** - words like Dual, Triple and Quad are used to represent number of cameras and front and rear cameras are separated by '&'
14. **camera** - problem with rows (100,113,151,157,161,238,273,308,309,323,324,365,367,378,394,441,450,484,506,534,553,571,572,575,584,610,613,615,630,642,647,649,659,667,684,687,705,711,723,728,750,756,759,764,792,819,846,854,855,858,883,884,896,915,916,927,929,932,945,956,990,995,1002,1016
) `validity`
15. **card** - sometimes contains info about os and camera `validity`
16. **os** - sometimes contains info about bluetooth and fm radio `validity`
17. **os** - issue with rows (324,378) `validity`
18. **os** - sometimes contains os version name like lollipop `consistency`
19. missing values in camera, card and os `completeness`
20. datatype  of price and rating is incorrect `validity`



### 1.2 Tidiness Issues

1. **sim** - can be split into 3 cols has_5g, has_NFC, has_IR_Blaster
2. **ram** - can be split into 2 cols RAM and ROM
3. **processor** - can be split into processor name, cores and cpu speed.
4. **battery** - can be split into battery capacity, fast_charging_available
5. **display** - can be split into size, resolution_width, resolution_height and frequency
6. **camera** - can be split into front and rear camera
7. **card** - can be split into supported, extended_upto

## 2. Preliminary Analysis

### 2.1 Check Data Types

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   model      1020 non-null   object 
 1   price      1020 non-null   object 
 2   rating     879 non-null    float64
 3   sim        1020 non-null   object 
 4   processor  1020 non-null   object 
 5   ram        1020 non-null   object 
 6   battery    1020 non-null   object 
 7   display    1020 non-null   object 
 8   camera     1019 non-null   object 
 9   card       1013 non-null   object 
 10  os         1003 non-null   object 
dtypes: float64(1), object(10)
memory usage: 87.8+ KB


### 2.2 Descriptive Statistics

In [8]:
df.describe()

,rating
count,879.000000
mean,78.258248
std,7.402854
min,60.000000
25%,74.000000
50%,80.000000
75%,84.000000
max,89.000000


### 2.3 Check for Duplicates

In [9]:
df.duplicated().sum()

np.int64(0)

## 3. Detailed Analysis (Column wise)

### 3.1 price

In [11]:
(
    df.price
    .str.replace('₹', '', regex=False)  
    .str.replace(',', '', regex=False)
    .astype("int")
    .describe()
)

count      1020.000000
mean      31371.767647
std       39168.942590
min          99.000000
25%       12464.250000
50%       19815.000000
75%       34999.000000
max      650000.000000
Name: price, dtype: float64

In [16]:
(
    df
    .pipe(adjust_index) 
    .reset_index()  
    .loc[
        lambda x: x['index'].isin(
            processor_rows
            .union(ram_rows)
            .union(battery_rows)
            .union(display_rows)
            .union(camera_rows)
        )
    ]
)


,level_0,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
98,98,100,Vivo X Fold 5G,"₹1,06,990",NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen1, Octa Core, 3 GHz Processor","12 GB RAM, 256 GB inbuilt",4600 mAh Battery with 66W Fast Charging,"8.03 inches, 1916 x 2160 px, 120 Hz Display",Foldable Display,50 MP Quad Rear & 16 MP Front Camera,Android v12
111,111,113,Apple iPhone 12,"₹51,999",74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt","6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
149,149,151,Apple iPhone 12 Mini,"₹40,999",74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt","5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
155,155,157,Nokia 2780 Flip,"₹4,990",NaN,"Dual Sim, 3G, 4G, Wi-Fi","Snapdragon QM215, Quad Core, 1.3 GHz Processor","4 GB RAM, 512 MB inbuilt",1450 mAh Battery,"2.7 inches, 240 x 320 px Display",Dual Display,5 MP Rear Camera,"Memory Card Supported, upto 32 GB"
159,159,161,Oppo Find N2 5G,"₹94,990",NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",4520 mAh Battery with 67W Fast Charging,"7.1 inches, 1792 x 1920 px, 120 Hz Display wit...","Foldable Display, Dual Display",50 MP + 48 MP + 32 MP Triple Rear & 32 MP + 32...,Memory Card Not Supported
...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,954,956,Vivo X Fold 5G (12GB RAM + 512GB),"₹1,18,990",NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen1, Octa Core, 3 GHz Processor","12 GB RAM, 512 GB inbuilt",4600 mAh Battery with 66W Fast Charging,"8.03 inches, 1916 x 2160 px, 120 Hz Display",Foldable Display,50 MP Quad Rear & 16 MP Front Camera,Android v12
988,988,990,Nokia 5310 Dual Sim,"₹3,399",NaN,Dual Sim,"8 MB RAM, 16 MB inbuilt",1200 mAh Battery,"2.4 inches, 240 x 320 px Display",0.3 MP Rear Camera,"Memory Card Supported, upto 32 GB",Bluetooth,Browser
993,993,995,Huawei Mate X,"₹1,69,000",NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...","Kirin 990, Octa Core, 2.86 GHz Processor","8 GB RAM, 512 GB inbuilt",4500 mAh Battery with 55W Fast Charging,"8 inches, 2200 x 2480 px Display",Foldable Display,48 MP Quad Rear Camera,"Memory Card (Hybrid), upto 256 GB"
1000,1000,1002,XTouch F40 Flip,"₹1,999",NaN,Dual Sim,No 3G,No Wifi,"32 MB RAM, 32 MB inbuilt",800 mAh Battery,"1.77 inches, 240 x 320 px Display",Dual Display,1.3 MP Rear Camera


In [17]:
(
    df
    .pipe(adjust_index)  
    .reset_index()  
    .loc[  
        lambda x: x['index'].isin(
            processor_rows
            & ram_rows
            & battery_rows
            & display_rows
            & camera_rows
        )
    ]
    # .shape
)


,level_0,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
640,640,642,Nokia 105 Plus,"₹1,299",NaN,Dual Sim,"4 MB RAM, 4 MB inbuilt",800 mAh Battery,"1.77 inches, 128 x 160 px Display",No Rear Camera,"Memory Card Supported, upto 32 GB",Bluetooth,NaN
645,645,647,Nokia 2760 Flip,"₹5,490",NaN,"Dual Sim, 3G, 4G, Wi-Fi",1450 mAh Battery,"3.6 inches, 240 x 320 px Display",5 MP Rear & 5 MP Front Camera,"Memory Card Supported, upto 32 GB",Kaios v3.0,Bluetooth,NaN
647,647,649,Motorola Moto A10,"₹1,339",NaN,Dual Sim,"4 MB RAM, 4 MB inbuilt",1750 mAh Battery,"1.8 inches, 160 x 128 px Display",No Rear Camera,"Memory Card Supported, upto 32 GB",NaN,NaN
657,657,659,Zanco Tiny T1,"₹2,799",NaN,Single Sim,"32 MB RAM, 32 MB inbuilt",200 mAh Battery,"0.49 inches, 64 x 32 px Display",No Rear Camera,No FM Radio,Bluetooth,NaN
665,665,667,itel it2163S,₹958,NaN,Dual Sim,"4 MB RAM, 4 MB inbuilt",1200 mAh Battery,"1.8 inches, 160 x 128 px Display",No Rear Camera,"Memory Card Supported, upto 32 GB",Bluetooth,NaN
748,748,750,Nokia 400 4G,"₹3,290",NaN,"Dual Sim, 4G, VoLTE, Wi-Fi",2000 mAh Battery,"2.4 inches, 240 x 320 px Display",0.3 MP Rear & 0.3 MP Front Camera,"Memory Card Supported, upto 64 GB",Bluetooth,Browser,NaN
757,757,759,Karbonn KU3i,₹995,NaN,Dual Sim,"52 MB RAM, 32 MB inbuilt",1000 mAh Battery,"1.8 inches, 128 x 160 px Display",No Rear Camera,"Memory Card Supported, upto 16 GB",Bluetooth,NaN
817,817,819,itel Magic X,"₹2,239",NaN,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi",No 3G,T117,"48 MB RAM, 128 MB inbuilt",1200 mAh Battery,"2.4 inches, 240 x 320 px Display",1.3 MP Rear Camera,"Memory Card Supported, upto 64 GB"
882,882,884,Nokia 5710 XpressAudio,"₹4,799",NaN,"Dual Sim, 3G, 4G",No Wifi,Unisoc T107,"48 MB RAM, 128 MB inbuilt",1450 mAh Battery,"2.4 inches, 240 x 320 px Display",0.3 MP Rear Camera,"Memory Card Supported, upto 32 GB"
925,925,927,Nokia 3310 4G,"₹3,999",NaN,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi","256 MB RAM, 512 MB inbuilt",1200 mAh Battery,"2.4 inches, 240 x 320 px Display",2 MP Rear Camera,"Memory Card Supported, upto 32 GB",Bluetooth,Browser


In [18]:
(
    df
    .assign(
        price = lambda df_: (
            df_.price
            .str.replace('₹', '', regex=False)  
            .str.replace(',', '', regex=False) 
            .astype('int')  
        )
    )
    .loc[lambda x: x['price'] >= 3400] 
    # .shape
)


,model,price,rating,sim,processor,ram,battery,display,camera,card,os
0,OnePlus 11 5G,54999,89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,Memory Card Not Supported,Android v13
1,OnePlus Nord CE 2 Lite 5G,19989,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 695, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
2,Samsung Galaxy A14 5G,16499,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Exynos 1330, Octa Core, 2.4 GHz Processor","4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,"Memory Card Supported, upto 1 TB",Android v13
3,Motorola Moto G62 5G,14999,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 695, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
4,Realme 10 Pro Plus,24999,82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Dimensity 1080, Octa Core, 2.6 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,Memory Card Not Supported,Android v13
...,...,...,...,...,...,...,...,...,...,...,...
1015,Motorola Moto Edge S30 Pro,34990,83.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 8 Gen1, Octa Core, 3 GHz Processor","8 GB RAM, 128 GB inbuilt",5000 mAh Battery with 68.2W Fast Charging,"6.67 inches, 1080 x 2460 px, 120 Hz Display wi...",64 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,Android v12,No FM Radio
1016,Honor X8 5G,14990,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 480+, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 22.5W Fast Charging,"6.5 inches, 720 x 1600 px Display with Water D...",48 MP + 2 MP + Depth Sensor Triple Rear & 8 MP...,"Memory Card Supported, upto 1 TB",Android v11
1017,POCO X4 GT 5G (8GB RAM + 256GB),28990,85.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...","Dimensity 8100, Octa Core, 2.85 GHz Processor","8 GB RAM, 256 GB inbuilt",5080 mAh Battery with 67W Fast Charging,"6.6 inches, 1080 x 2460 px, 144 Hz Display wit...",64 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,Memory Card Not Supported,Android v12
1018,Motorola Moto G91 5G,19990,80.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 695, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,"6.8 inches, 1080 x 2400 px Display with Punch ...",108 MP + 8 MP + 2 MP Triple Rear & 32 MP Front...,"Memory Card Supported, upto 1 TB",Android v12


### 3.2 processor

In [19]:
(
    df
    .assign(
        price = lambda df_: (
            df_.price
            .str.replace('₹', '', regex=False)  
            .str.replace(',', '', regex=False) 
            .astype('int')  
        )
    )
    .loc[lambda x: x['price'] >= 3400] 
    .pipe(adjust_index)  
    .reset_index()  
    .loc[  
        lambda x: x['index'].isin(
            processor_rows          
        )
    ]
    # .shape
)

,level_0,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
635,635,647,Nokia 2760 Flip,5490,NaN,"Dual Sim, 3G, 4G, Wi-Fi",1450 mAh Battery,"3.6 inches, 240 x 320 px Display",5 MP Rear & 5 MP Front Camera,"Memory Card Supported, upto 32 GB",Kaios v3.0,Bluetooth,NaN
836,836,859,LG Folder 2,11999,NaN,"Single Sim, 3G, 4G, Wi-Fi","1 GB RAM, 8 GB inbuilt",1470 mAh Battery,"2.8 inches, 240 x 320 px Display",2 MP Rear Camera,Memory Card Supported,Bluetooth,NaN
859,859,884,Nokia 5710 XpressAudio,4799,NaN,"Dual Sim, 3G, 4G",No Wifi,Unisoc T107,"48 MB RAM, 128 MB inbuilt",1450 mAh Battery,"2.4 inches, 240 x 320 px Display",0.3 MP Rear Camera,"Memory Card Supported, upto 32 GB"
901,901,927,Nokia 3310 4G,3999,NaN,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi","256 MB RAM, 512 MB inbuilt",1200 mAh Battery,"2.4 inches, 240 x 320 px Display",2 MP Rear Camera,"Memory Card Supported, upto 32 GB",Bluetooth,Browser


### 3.3 ram

In [34]:
(
    clean_data(df)  
    .loc[  
        lambda x: x['index'].isin(
            ram_rows          
        )
    ]
    # .shape
)

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
439,441,Apple iPhone SE 3 2022,43900,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A15, Hexa Core, 3.22 GHz Processor",64 GB inbuilt,"4.7 inches, 750 x 1334 px Display",12 MP Rear & 7 MP Front Camera,Memory Card Not Supported,iOS v15,No FM Radio
483,485,Huawei Mate 50 RS Porsche Design,239999,81.0,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi, NFC, IR Blaster","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor",512 GB inbuilt,4700 mAh Battery with 66W Fast Charging,"6.74 inches, 1212 x 2616 px, 120 Hz Display",50 MP + 48 MP + 13 MP Triple Rear & 13 MP Fron...,"Memory Card (Hybrid), upto 256 GB",Hongmeng OS v3.0
582,584,Nokia 8210 4G,3749,NaN,"Dual Sim, 3G, 4G",No Wifi,Unisoc T107,"48 MB RAM, 128 MB inbuilt",1450 mAh Battery,"2.8 inches, 240 x 320 px Display",0.3 MP Rear Camera,"Memory Card Supported, upto 32 GB"


### 3.4 battery

In [48]:
(
    clean_data(df)  
    .loc[  
        lambda x: x['index'].isin(
            battery_rows          
        )
    ]
    # .shape
)

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
111,113,Apple iPhone 12,51999,74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt","6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
149,151,Apple iPhone 12 Mini,40999,74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt","5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
307,309,Apple iPhone 12 (128GB),55999,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 128 GB inbuilt","6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
363,365,Apple iPhone 12 Mini (128GB),45999,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 128 GB inbuilt","5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
439,441,Apple iPhone SE 3 2022,43900,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A15, Hexa Core, 3.22 GHz Processor",64 GB inbuilt,"4.7 inches, 750 x 1334 px Display",12 MP Rear & 7 MP Front Camera,Memory Card Not Supported,iOS v15,No FM Radio
448,450,Apple iPhone 15 Pro,130990,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",Bionic A16,"8 GB RAM, 128 GB inbuilt","6.06 inches, 1170 x 2532 px, 120 Hz Display wi...",50 MP + 12 MP + 12 MP Triple Rear & 12 MP Fron...,Memory Card Not Supported,iOS v15,No FM Radio
628,630,Apple iPhone 12 Pro (512GB),139900,80.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","6 GB RAM, 512 GB inbuilt","6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP + 12 MP Triple Rear & 12 MP Fron...,Memory Card Not Supported,iOS v14.0,No FM Radio
762,764,Apple iPhone SE 4,49990,60.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A15, Hexa Core, 3.22 GHz Processor",64 GB inbuilt,"6.1 inches, 750 x 1580 px Display",12 MP Rear & 10.8 MP Front Camera,Memory Card Not Supported,iOS v16,No FM Radio
853,855,Apple iPhone 12 Pro (256GB),119900,80.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","6 GB RAM, 256 GB inbuilt","6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP + 12 MP Triple Rear & 12 MP Fron...,Memory Card Not Supported,iOS v14.0,No FM Radio
913,915,Apple iPhone 12 Mini (256GB),55999,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 256 GB inbuilt","5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio


### 3.5 display

In [56]:
(
    clean_data(df)  
    .loc[  
        lambda x: x['index'].isin(
            display_rows          
        )
    ]
    # .shape
)

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
439,441,Apple iPhone SE 3 2022,43900,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A15, Hexa Core, 3.22 GHz Processor",64 GB inbuilt,None,"4.7 inches, 750 x 1334 px Display",12 MP Rear & 7 MP Front Camera,Memory Card Not Supported,iOS v15
448,450,Apple iPhone 15 Pro,130990,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",Bionic A16,"8 GB RAM, 128 GB inbuilt",None,"6.06 inches, 1170 x 2532 px, 120 Hz Display wi...",50 MP + 12 MP + 12 MP Triple Rear & 12 MP Fron...,Memory Card Not Supported,iOS v15
628,630,Apple iPhone 12 Pro (512GB),139900,80.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","6 GB RAM, 512 GB inbuilt",None,"6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP + 12 MP Triple Rear & 12 MP Fron...,Memory Card Not Supported,iOS v14.0
762,764,Apple iPhone SE 4,49990,60.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A15, Hexa Core, 3.22 GHz Processor",64 GB inbuilt,None,"6.1 inches, 750 x 1580 px Display",12 MP Rear & 10.8 MP Front Camera,Memory Card Not Supported,iOS v16
913,915,Apple iPhone 12 Mini (256GB),55999,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 256 GB inbuilt",None,"5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14
914,916,Apple iPhone 12 (256GB),67999,76.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 256 GB inbuilt",None,"6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14


In [57]:
len(display_rows)

27

In [58]:
len(camera_rows)

64

### 3.6 camera

In [60]:
(
    clean_data(df)  
    .loc[  
        lambda x: x['index'].isin(
            camera_rows          
        )
    ]
    # .shape
)

# 155 271

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
98,100,Vivo X Fold 5G,106990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen1, Octa Core, 3 GHz Processor","12 GB RAM, 256 GB inbuilt",NaN,NaN,NaN,NaN,NaN
111,113,Apple iPhone 12,51999,74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt",None,"6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14
149,151,Apple iPhone 12 Mini,40999,74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt",None,"5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14
155,157,Nokia 2780 Flip,4990,NaN,"Dual Sim, 3G, 4G, Wi-Fi","Snapdragon QM215, Quad Core, 1.3 GHz Processor","4 GB RAM, 512 MB inbuilt",NaN,NaN,NaN,NaN,NaN
159,161,Oppo Find N2 5G,94990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",NaN,NaN,NaN,NaN,NaN
236,238,Xiaomi Mix Fold 2 5G,106990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...","Snapdragon 8+ Gen1 , Octa Core, 3.2 GHz Proce...","12 GB RAM, 256 GB inbuilt",NaN,NaN,NaN,NaN,NaN
271,273,Nokia 2720 V Flip,6199,NaN,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi","Snapdragon 205 , Dual Core, 1.1 GHz Processor","512 MB RAM, 4 GB inbuilt",NaN,NaN,NaN,NaN,NaN
306,308,Samsung Galaxy Z Flip 3,69999,84.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 888, Octa Core, 2.84 GHz Processor","8 GB RAM, 128 GB inbuilt",NaN,NaN,NaN,NaN,NaN
307,309,Apple iPhone 12 (128GB),55999,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 128 GB inbuilt",None,"6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14
321,323,Samsung Galaxy Z Fold 4,154998,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",NaN,NaN,NaN,NaN,NaN


### 3.7 card

In [47]:
(
    clean_data(df)
    .card
    .value_counts()
)

card
Memory Card Not Supported              362
Memory Card Supported, upto 1 TB       171
Memory Card Supported, upto 512 GB     105
Memory Card (Hybrid), upto 1 TB         91
Memory Card Supported                   89
Memory Card Supported, upto 256 GB      87
Memory Card (Hybrid)                    30
Memory Card (Hybrid), upto 256 GB       13
Memory Card (Hybrid), upto 512 GB       11
Memory Card Supported, upto 128 GB       6
Memory Card Supported, upto 2 TB         5
Memory Card Supported, upto 32 GB        4
Memory Card (Hybrid), upto 128 GB        3
Memory Card (Hybrid), upto 64 GB         3
Memory Card Supported, upto 1000 GB      1
Memory Card (Hybrid), upto 2 TB          1
Name: count, dtype: int64

### 3.8 os

In [57]:
(
    clean_data(df)
    .os
    .value_counts()
)

os
Android v12                           394
Android v11                           274
Android v13                            91
Android v10                            69
Android v9.0 (Pie)                     29
Android v10.0                          23
iOS v16                                15
iOS v15                                12
Android v8.1 (Oreo)                    10
iOS v14                                 6
Memory Card Not Supported               6
Android v8.0 (Oreo)                     4
Android v11.0                           4
iOS v13                                 4
iOS v15.0                               3
Memory Card (Hybrid), upto 2 TB         2
Memory Card Supported, upto 256 GB      2
Android v5.1.1 (Lollipop)               2
Android v12.1                           2
HarmonyOS v2.0                          2
iOS v14.0                               2
EMUI v12                                2
Android v6.0 (Marshmallow)              2
Memory Card (Hybrid), upto 256 

In [61]:
(
    clean_data(df)
    .os
    .value_counts()
)

os
Android v12                           394
Android v11                           274
Android v13                            91
Android v10                            69
Android v9.0 (Pie)                     29
Android v10.0                          23
iOS v16                                15
iOS v15                                12
Android v8.1 (Oreo)                    10
iOS v14                                 6
Memory Card Not Supported               6
Android v11.0                           4
iOS v13                                 4
Android v8.0 (Oreo)                     4
iOS v15.0                               3
Android v5.1.1 (Lollipop)               2
Memory Card Supported, upto 256 GB      2
Android v12.1                           2
HarmonyOS v2.0                          2
Memory Card (Hybrid), upto 2 TB         2
EMUI v12                                2
Android v6.0 (Marshmallow)              2
Memory Card (Hybrid), upto 256 GB       2
iOS v14.0                      

### 3.9 display

In [64]:
(
    clean_data(df)
    .display
    .value_counts()
)

display
6.67 inches, 1080 x 2400 px, 120 Hz Display with Punch Hole    54
6.5 inches, 720 x 1600 px Display with Water Drop Notch        36
6.7 inches, 1080 x 2412 px, 120 Hz Display with Punch Hole     25
6.52 inches, 720 x 1600 px Display with Water Drop Notch       23
6.5 inches, 1080 x 2400 px, 90 Hz Display with Punch Hole      22
                                                               ..
5.86 inches, 720 x 1520 px Display with Large Notch             1
6.43 inches, 1440 x 3200 px, 120 Hz Display with Punch Hole     1
6.6 inches, 1080 x 2400 px, 144 Hz Display                      1
6.73 inches, 1440 x 3120 px, 120 Hz Display with Punch Hole     1
6.8 inches, 1080 x 2400 px Display with Punch Hole              1
Name: count, Length: 347, dtype: int64

In [ ]:
(982/1020)*100

96.27450980392157

In [ ]:
has_5g = df1['sim'].str.contains('5G')
has_nfc = df1['sim'].str.contains('NFC')
has_ir_blaster = df1['sim'].str.contains('IR Blaster')

In [ ]:
df1.insert(6,'has_5g',has_5g)
df1.insert(7,'has_nfc',has_nfc)
df1.insert(8,'has_ir_blaster',has_ir_blaster)

In [ ]:
processor_name = df1['processor'].str.split(',').str.get(0)

In [ ]:
num_cores = df1['processor'].str.split(',').str.get(1)

In [ ]:
processor_speed = df1['processor'].str.split(',').str.get(2)

In [ ]:
df1.insert(10,'processor_name',processor_name)
df1.insert(11,'num_cores',num_cores)
df1.insert(12,'processor_speed',processor_speed)

In [ ]:
df1['processor_name'] = df1['processor_name'].str.strip()

In [ ]:
temp_df = df1[df1['processor_name'].str.contains('Core')][['processor_name', 'num_cores',	'processor_speed']].shift(1,axis=1)

In [ ]:
temp_df.shape

(20, 3)

In [ ]:
df1.loc[temp_df.index,['processor_name', 'num_cores',	'processor_speed']] = temp_df.values

In [ ]:
df1.loc[856]

index                                                858
brand_name                                       samsung
model                            Samsung Galaxy A01 Core
price                                               4999
rating                                               NaN
sim                       Dual Sim, 3G, 4G, VoLTE, Wi-Fi
has_5g                                             False
has_nfc                                            False
has_ir_blaster                                     False
processor          (28 nm), Quad Core, 1.5 GHz Processor
processor_name                                   (28 nm)
num_cores                                      Quad Core
processor_speed                        1.5 GHz Processor
ram                              1 GB RAM, 16 GB inbuilt
battery                                 3000 mAh Battery
display                5.3 inches, 720 x 1480 px Display
camera                     8 MP Rear & 5 MP Front Camera
card                           

In [ ]:
df1.loc[856,'processor_name'] = 'Mediatek MT6739'

In [ ]:
processor_brand = df1['processor_name'].str.split(' ').str.get(0).str.lower()

In [ ]:
df1.insert(11,'processor_brand',processor_brand)

In [ ]:
df1['num_cores'] = df1['num_cores'].str.strip()

In [ ]:
df1['num_cores'] = df1['num_cores'].str.replace('Octa Core Processor','Octa Core').str.replace('Hexa Core Processor','Hexa Core')

In [ ]:
df1['processor_speed'] = df1['processor_speed'].str.strip().str.split(' ').str.get(0).str.replace('\u2009',' ').str.split(' ').str.get(0).astype(float)

In [ ]:
df1.head()

,index,brand_name,model,price,rating,sim,has_5g,has_nfc,has_ir_blaster,processor,processor_name,processor_brand,num_cores,processor_speed,ram,battery,display,camera,card,os
0,2,oneplus,OnePlus 11 5G,54999,89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor",Snapdragon 8 Gen2,snapdragon,Octa Core,3.2,"12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,Memory Card Not Supported,Android v13
1,3,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
2,4,samsung,Samsung Galaxy A14 5G,16499,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Exynos 1330, Octa Core, 2.4 GHz Processor",Exynos 1330,exynos,Octa Core,2.4,"4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,"Memory Card Supported, upto 1 TB",Android v13
3,5,motorola,Motorola Moto G62 5G,14999,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
4,6,realme,Realme 10 Pro Plus,24999,82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Dimensity 1080, Octa Core, 2.6 GHz Processor",Dimensity 1080,dimensity,Octa Core,2.6,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,Memory Card Not Supported,Android v13


In [ ]:
ram_capacity = df1['ram'].str.strip().str.split(',').str.get(0).str.findall(r'\b(\d+)\b').str.get(0)

In [ ]:
df1.insert(16,'ram_capacity',ram_capacity)

In [ ]:
df1.head()

,index,brand_name,model,price,rating,sim,has_5g,has_nfc,has_ir_blaster,processor,processor_name,processor_brand,num_cores,processor_speed,ram,battery,ram_capacity,display,camera,card,os
0,2,oneplus,OnePlus 11 5G,54999,89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor",Snapdragon 8 Gen2,snapdragon,Octa Core,3.2,"12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,12,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,Memory Card Not Supported,Android v13
1,3,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,6,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
2,4,samsung,Samsung Galaxy A14 5G,16499,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Exynos 1330, Octa Core, 2.4 GHz Processor",Exynos 1330,exynos,Octa Core,2.4,"4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,4,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,"Memory Card Supported, upto 1 TB",Android v13
3,5,motorola,Motorola Moto G62 5G,14999,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,6,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
4,6,realme,Realme 10 Pro Plus,24999,82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Dimensity 1080, Octa Core, 2.6 GHz Processor",Dimensity 1080,dimensity,Octa Core,2.6,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,6,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,Memory Card Not Supported,Android v13


In [ ]:
internal_memory = df1['ram'].str.strip().str.split(',').str.get(1).str.strip().str.findall(r'\b(\d+)\b').str.get(0)

In [ ]:
df1.insert(17,'internal_memory',internal_memory)

In [ ]:
df1['ram_capacity'] = df1['ram_capacity'].astype(float)

In [ ]:
df1.drop([486,627],inplace=True)

In [ ]:
df1.loc[[483], ['ram_capacity','internal_memory']] = [12.0,'512']

In [ ]:
df1['ram_capacity'].value_counts()

8.0     339
6.0     234
4.0     215
12.0     86
3.0      54
2.0      32
16.0      9
1.0       7
18.0      2
64.0      2
Name: ram_capacity, dtype: int64

In [ ]:
df1['internal_memory'] = df1['internal_memory'].astype(float)

In [ ]:
temp_df = df1[df1['internal_memory'] == 1]

In [ ]:
df1.loc[temp_df.index,'internal_memory'] = 1024

In [ ]:
df1['internal_memory'].value_counts()

128.0     523
64.0      191
256.0     157
32.0       67
512.0      22
16.0       12
1024.0      5
8.0         1
Name: internal_memory, dtype: int64

In [ ]:
battery_capacity = df1['battery'].str.strip().str.split('with').str.get(0).str.strip().str.findall(r'\b(\d+)\b').str.get(0).astype(float)

In [ ]:
df1.insert(16,'battery_capacity',battery_capacity)

In [ ]:
fast_charging = df1['battery'].str.strip().str.split('with').str.get(1).str.strip().str.findall(r'\d{2,3}')

In [ ]:
df1.insert(17,'fast_charging',fast_charging)

In [ ]:
def fast_charging_extractor(item):

  if type(item) == list:
    if len(item) == 1:
      return item[0]
    else:
      return 0
  else:
    return -1

In [ ]:
df1['fast_charging'] = df1['fast_charging'].apply(fast_charging_extractor).astype(int)

In [ ]:
screen_size = df1['display'].str.strip().str.split(',').str.get(0).str.strip().str.split(' ').str.get(0).astype(float)

In [ ]:
df1.insert(21,'screen_size',screen_size)

In [ ]:
resolution = df1['display'].str.strip().str.split(',').str.get(1).str.strip().str.split('px').str.get(0)

In [ ]:
df1.insert(22,'resolution',resolution)

In [ ]:
refresh_rate = df1['display'].str.strip().str.split(',').str.get(2).str.strip().str.findall(r'\d{2,3}').str.get(0).apply(lambda x: 60 if pd.isna(x) else x).astype(int)

In [ ]:
df1.insert(22,'refresh_rate',refresh_rate)

In [ ]:
df1.head()

,index,brand_name,model,price,rating,sim,has_5g,has_nfc,has_ir_blaster,processor,processor_name,processor_brand,num_cores,processor_speed,ram,battery,battery_capacity,fast_charging,ram_capacity,internal_memory,display,screen_size,refresh_rate,resolution,camera,card,os
0,2,oneplus,OnePlus 11 5G,54999,89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor",Snapdragon 8 Gen2,snapdragon,Octa Core,3.2,"12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,5000.0,100,12.0,256.0,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",6.70,120,1440 x 3216,50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,Memory Card Not Supported,Android v13
1,3,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,5000.0,33,6.0,128.0,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",6.59,120,1080 x 2412,64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
2,4,samsung,Samsung Galaxy A14 5G,16499,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Exynos 1330, Octa Core, 2.4 GHz Processor",Exynos 1330,exynos,Octa Core,2.4,"4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,5000.0,15,4.0,64.0,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",6.60,90,1080 x 2408,50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,"Memory Card Supported, upto 1 TB",Android v13
3,5,motorola,Motorola Moto G62 5G,14999,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,5000.0,0,6.0,128.0,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",6.55,120,1080 x 2400,50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
4,6,realme,Realme 10 Pro Plus,24999,82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Dimensity 1080, Octa Core, 2.6 GHz Processor",Dimensity 1080,dimensity,Octa Core,2.6,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,5000.0,67,6.0,128.0,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",6.70,120,1080 x 2412,108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,Memory Card Not Supported,Android v13


In [ ]:
def camera_extractor(text):

  if 'Quad' in text:
    return '4'
  elif 'Triple' in text:
    return '3'
  elif 'Dual' in text:
    return '2'
  elif 'Missing' in text:
    return 'Missing'
  else:
    return '1'

In [ ]:
num_rear_cameras = df1['camera'].str.strip().str.split('&').str.get(0).apply(camera_extractor)

In [ ]:
df1.insert(25,'num_rear_cameras',num_rear_cameras)

In [ ]:
num_front_cameras = df1['camera'].str.strip().str.split('&').str.get(1).str.strip().fillna('Missing').apply(camera_extractor)

In [ ]:
df1.insert(26,'num_front_cameras',num_front_cameras)

In [ ]:
df1.head()

,index,brand_name,model,price,rating,sim,has_5g,has_nfc,has_ir_blaster,processor,processor_name,processor_brand,num_cores,processor_speed,ram,battery,battery_capacity,fast_charging,ram_capacity,internal_memory,display,screen_size,refresh_rate,resolution,camera,num_rear_cameras,num_front_cameras,card,os
0,2,oneplus,OnePlus 11 5G,54999,89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor",Snapdragon 8 Gen2,snapdragon,Octa Core,3.2,"12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,5000.0,100,12.0,256.0,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",6.70,120,1440 x 3216,50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,3,1,Memory Card Not Supported,Android v13
1,3,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,5000.0,33,6.0,128.0,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",6.59,120,1080 x 2412,64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,"Memory Card (Hybrid), upto 1 TB",Android v12
2,4,samsung,Samsung Galaxy A14 5G,16499,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Exynos 1330, Octa Core, 2.4 GHz Processor",Exynos 1330,exynos,Octa Core,2.4,"4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,5000.0,15,4.0,64.0,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",6.60,90,1080 x 2408,50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,3,1,"Memory Card Supported, upto 1 TB",Android v13
3,5,motorola,Motorola Moto G62 5G,14999,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,5000.0,0,6.0,128.0,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",6.55,120,1080 x 2400,50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,"Memory Card (Hybrid), upto 1 TB",Android v12
4,6,realme,Realme 10 Pro Plus,24999,82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Dimensity 1080, Octa Core, 2.6 GHz Processor",Dimensity 1080,dimensity,Octa Core,2.6,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,5000.0,67,6.0,128.0,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",6.70,120,1080 x 2412,108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,3,1,Memory Card Not Supported,Android v13


In [ ]:
df1[df1['num_front_cameras'] == 'Missing']

,index,brand_name,model,price,rating,sim,has_5g,has_nfc,has_ir_blaster,processor,processor_name,processor_brand,num_cores,processor_speed,ram,battery,battery_capacity,fast_charging,ram_capacity,internal_memory,display,screen_size,refresh_rate,resolution,camera,num_rear_cameras,num_front_cameras,card,os
69,71,oppo,Oppo Find N Fold,99990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor",Snapdragon 8+ Gen1,snapdragon,Octa Core,3.20,"8 GB RAM, 256 GB inbuilt",5000 mAh Battery with 67W Fast Charging,5000.0,67,8.0,256.0,"7.1 inches, 1792 x 1920 px, 120 Hz Display wit...",7.10,120,1792 x 1920,"Foldable Display, Dual Display",2,Missing,Memory Card Not Supported,Android v12
462,464,xiaomi,Xiaomi Mi Mix Alpha,199990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 855+, Octa Core, 2.96 GHz Processor",Snapdragon 855+,snapdragon,Octa Core,2.96,"12 GB RAM, 512 GB inbuilt",4050 mAh Battery with 40W Fast Charging,4050.0,40,12.0,512.0,"7.92 inches, 2088 x 2250 px Display",7.92,60,2088 x 2250,108 MP + 20 MP + 12 MP Triple Rear Camera,3,Missing,Memory Card Not Supported,Android v10
993,995,huawei,Huawei Mate X,169000,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...",True,True,True,"Kirin 990, Octa Core, 2.86 GHz Processor",Kirin 990,kirin,Octa Core,2.86,"8 GB RAM, 512 GB inbuilt",4500 mAh Battery with 55W Fast Charging,4500.0,55,8.0,512.0,"8 inches, 2200 x 2480 px Display",8.00,60,2200 x 2480,48 MP Quad Rear Camera,4,Missing,Memory Card Not Supported,"Memory Card (Hybrid), upto 256 GB"
1012,1014,itel,itel A23s,4787,NaN,"Dual Sim, 3G, 4G, Wi-Fi",False,False,False,"Spreadtrum SC9832E, Quad Core, 1.4 GHz Processor",Spreadtrum SC9832E,spreadtrum,Quad Core,1.40,"2 GB RAM, 32 GB inbuilt",3020 mAh Battery,3020.0,-1,2.0,32.0,"5 inches, 854 x 480 px Display",5.00,60,854 x 480,2 MP Rear Camera,1,Missing,Memory Card Not Supported,Android v11


In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
df1.head()

,index,brand_name,model,price,rating,sim,has_5g,has_nfc,has_ir_blaster,processor,processor_name,processor_brand,num_cores,processor_speed,ram,battery,battery_capacity,fast_charging,ram_capacity,internal_memory,display,screen_size,refresh_rate,resolution,camera,num_rear_cameras,num_front_cameras,card,os
0,2,oneplus,OnePlus 11 5G,54999,89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor",Snapdragon 8 Gen2,snapdragon,Octa Core,3.2,"12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,5000.0,100,12.0,256.0,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",6.70,120,1440 x 3216,50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,3,1,Memory Card Not Supported,Android v13
1,3,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,5000.0,33,6.0,128.0,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",6.59,120,1080 x 2412,64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,"Memory Card (Hybrid), upto 1 TB",Android v12
2,4,samsung,Samsung Galaxy A14 5G,16499,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Exynos 1330, Octa Core, 2.4 GHz Processor",Exynos 1330,exynos,Octa Core,2.4,"4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,5000.0,15,4.0,64.0,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",6.60,90,1080 x 2408,50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,3,1,"Memory Card Supported, upto 1 TB",Android v13
3,5,motorola,Motorola Moto G62 5G,14999,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,5000.0,0,6.0,128.0,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",6.55,120,1080 x 2400,50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,"Memory Card (Hybrid), upto 1 TB",Android v12
4,6,realme,Realme 10 Pro Plus,24999,82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Dimensity 1080, Octa Core, 2.6 GHz Processor",Dimensity 1080,dimensity,Octa Core,2.6,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,5000.0,67,6.0,128.0,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",6.70,120,1080 x 2412,108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,3,1,Memory Card Not Supported,Android v13


In [ ]:
#df1[df1['camera'] == 'Foldable Display, Dual Display']
df1.loc[69,'camera'] == '50 MP'

False

In [ ]:
temp_df = df1[df1['camera'] == 'Foldable Display, Dual Display']

In [ ]:
df1.loc[temp_df.index, 'camera'] = '50 MP'

In [ ]:
df1['primary_camera_rear'] = df1['camera'].str.split(' ').str.get(0).str.replace('\u2009',' ').str.split(' ').str.get(0)

In [ ]:
df1['primary_camera_front'] = df1['camera'].str.split('&').str.get(1).str.strip().str.split(' ').str.get(0).str.replace('\u2009',' ').str.split(' ').str.get(0)

In [ ]:
df1.head()

,index,brand_name,model,price,rating,sim,has_5g,has_nfc,has_ir_blaster,processor,processor_name,processor_brand,num_cores,processor_speed,ram,battery,battery_capacity,fast_charging,ram_capacity,internal_memory,display,screen_size,refresh_rate,resolution,camera,num_rear_cameras,num_front_cameras,card,os,primary_camera_rear,primary_camera_front
0,2,oneplus,OnePlus 11 5G,54999,89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor",Snapdragon 8 Gen2,snapdragon,Octa Core,3.2,"12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,5000.0,100,12.0,256.0,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",6.70,120,1440 x 3216,50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,3,1,Memory Card Not Supported,Android v13,50,16
1,3,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,5000.0,33,6.0,128.0,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",6.59,120,1080 x 2412,64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,"Memory Card (Hybrid), upto 1 TB",Android v12,64,16
2,4,samsung,Samsung Galaxy A14 5G,16499,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Exynos 1330, Octa Core, 2.4 GHz Processor",Exynos 1330,exynos,Octa Core,2.4,"4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,5000.0,15,4.0,64.0,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",6.60,90,1080 x 2408,50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,3,1,"Memory Card Supported, upto 1 TB",Android v13,50,13
3,5,motorola,Motorola Moto G62 5G,14999,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,5000.0,0,6.0,128.0,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",6.55,120,1080 x 2400,50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,"Memory Card (Hybrid), upto 1 TB",Android v12,50,16
4,6,realme,Realme 10 Pro Plus,24999,82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Dimensity 1080, Octa Core, 2.6 GHz Processor",Dimensity 1080,dimensity,Octa Core,2.6,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,5000.0,67,6.0,128.0,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",6.70,120,1080 x 2412,108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,3,1,Memory Card Not Supported,Android v13,108,16


In [ ]:
df1[df1['card'] == 'Memory Card (Hybrid)']

,index,brand_name,model,price,rating,sim,has_5g,has_nfc,has_ir_blaster,processor,processor_name,processor_brand,num_cores,processor_speed,ram,battery,battery_capacity,fast_charging,ram_capacity,internal_memory,display,screen_size,refresh_rate,resolution,camera,num_rear_cameras,num_front_cameras,card,os,primary_camera_rear,primary_camera_front
25,27,samsung,Samsung Galaxy S20 FE 5G,31239,88.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 865, Octa Core, 2.84 GHz Processor",Snapdragon 865,snapdragon,Octa Core,2.84,"8 GB RAM, 128 GB inbuilt",4500 mAh Battery with Fast Charging,4500.0,0,8.0,128.0,"6.5 inches, 1080 x 2400 px, 120 Hz Display wit...",6.50,120,1080 x 2400,12 MP + 12 MP + 8 MP Triple Rear & 32 MP Front...,3,1,Memory Card (Hybrid),Android v10,12,32
80,82,motorola,Motorola Moto G72 4G,15999,83.0,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi, NFC",False,True,False,"Helio G99, Octa Core, 2.2 GHz Processor",Helio G99,helio,Octa Core,2.20,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,5000.0,33,6.0,128.0,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",6.55,120,1080 x 2400,108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,3,1,Memory Card (Hybrid),Android v12,108,16
147,149,samsung,Samsung Galaxy M34 5G,16999,84.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Exynos 1200, Octa Core, 2.4 GHz Processor",Exynos 1200,exynos,Octa Core,2.40,"6 GB RAM, 128 GB inbuilt",6000 mAh Battery with 25W Fast Charging,6000.0,25,6.0,128.0,"6.5 inches, 1080 x 2400 px, 120 Hz Display wit...",6.50,120,1080 x 2400,64 MP Quad Rear & 32 MP Front Camera,4,1,Memory Card (Hybrid),Android v12,64,32
151,153,motorola,Motorola Moto G52,11999,75.0,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi, NFC",False,True,False,"Snapdragon 680, Octa Core, 2.4 GHz Processor",Snapdragon 680,snapdragon,Octa Core,2.40,"4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 33W Fast Charging,5000.0,33,4.0,64.0,"6.6 inches, 2460 x 1080 px, 90 Hz Display with...",6.60,90,2460 x 1080,50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,Memory Card (Hybrid),Android v12,50,16
178,180,vivo,Vivo Y75 5G,21990,82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Dimensity 700 5G, Octa Core, 2.2 GHz Processor",Dimensity 700 5G,dimensity,Octa Core,2.20,"8 GB RAM, 128 GB inbuilt",5000 mAh Battery with 18W Fast Charging,5000.0,18,8.0,128.0,"6.58 inches, 1080 x 2408 px Display with Water...",6.58,60,1080 x 2408,50 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,Memory Card (Hybrid),Android v12,50,16
204,206,motorola,Motorola Moto G52 (6GB RAM + 128GB),12999,79.0,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi, NFC",False,True,False,"Snapdragon 680, Octa Core, 2.4 GHz Processor",Snapdragon 680,snapdragon,Octa Core,2.40,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,5000.0,33,6.0,128.0,"6.6 inches, 2460 x 1080 px, 90 Hz Display with...",6.60,90,2460 x 1080,50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,Memory Card (Hybrid),Android v12,50,16
248,250,motorola,Motorola Moto G73,25999,85.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Dimensity 930, Octa Core, 2.2 GHz Processor",Dimensity 930,dimensity,Octa Core,2.20,"8 GB RAM, 256 GB inbuilt",5000 mAh Battery with 30W Fast Charging,5000.0,30,8.0,256.0,"6.5 inches, 1080 x 2400 px, 120 Hz Display wit...",6.50,120,1080 x 2400,50 MP + 8 MP Dual Rear & 16 MP Front Camera,2,1,Memory Card (Hybrid),Android v13,50,16
254,256,xiaomi,Xiaomi Redmi Note 11 Pro 5G,17990,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, IR Blaster",True,False,True,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.20,"6 GB RAM, 64 GB inbuilt",5000 mAh Battery with 67W Fast Charging,5000.0,67,6.0,64.0,"6.67 inches, 1080 x 2400 px, 120 Hz Display wi...",6.67,120,1080 x 2400,108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,3,1,Memory Card (Hybrid),Android v11,108,16
281,283,vivo,Vivo V23e 5G,21994,84.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Dimensity 810 5G, Octa Core, 2.4 GHz P

In [ ]:
df1.loc[temp_df.index, 'card'] = 'Not Specified'

In [ ]:
df1['extended_memory'] = df1['card'].apply(lambda x:'0' if 'Not' in x else x.split('upto')).str.get(-1).str.strip().str.replace('Memory Card Supported','Not Specified')

In [ ]:
df1.head()

,index,brand_name,model,price,rating,sim,has_5g,has_nfc,has_ir_blaster,processor,processor_name,processor_brand,num_cores,processor_speed,ram,battery,battery_capacity,fast_charging,ram_capacity,internal_memory,display,screen_size,refresh_rate,resolution,camera,num_rear_cameras,num_front_cameras,card,os,primary_camera_rear,primary_camera_front,extended_memory
0,2,oneplus,OnePlus 11 5G,54999,89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor",Snapdragon 8 Gen2,snapdragon,Octa Core,3.2,"12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,5000.0,100,12.0,256.0,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",6.70,120,1440 x 3216,50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,3,1,Memory Card Not Supported,Android v13,50,16,0
1,3,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,5000.0,33,6.0,128.0,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",6.59,120,1080 x 2412,64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,"Memory Card (Hybrid), upto 1 TB",Android v12,64,16,1 TB
2,4,samsung,Samsung Galaxy A14 5G,16499,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Exynos 1330, Octa Core, 2.4 GHz Processor",Exynos 1330,exynos,Octa Core,2.4,"4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,5000.0,15,4.0,64.0,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",6.60,90,1080 x 2408,50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,3,1,"Memory Card Supported, upto 1 TB",Android v13,50,13,1 TB
3,5,motorola,Motorola Moto G62 5G,14999,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 695, Octa Core, 2.2 GHz Processor",Snapdragon 695,snapdragon,Octa Core,2.2,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,5000.0,0,6.0,128.0,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",6.55,120,1080 x 2400,50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,"Memory Card (Hybrid), upto 1 TB",Android v12,50,16,1 TB
4,6,realme,Realme 10 Pro Plus,24999,82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Dimensity 1080, Octa Core, 2.6 GHz Processor",Dimensity 1080,dimensity,Octa Core,2.6,"6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,5000.0,67,6.0,128.0,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",6.70,120,1080 x 2412,108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,3,1,Memory Card Not Supported,Android v13,108,16,0


In [ ]:
df1[df1['os'].str.contains('Memory Card')]

,index,brand_name,model,price,rating,sim,has_5g,has_nfc,has_ir_blaster,processor,processor_name,processor_brand,num_cores,processor_speed,ram,battery,battery_capacity,fast_charging,ram_capacity,internal_memory,display,screen_size,refresh_rate,resolution,camera,num_rear_cameras,num_front_cameras,card,os,primary_camera_rear,primary_camera_front,extended_memory
159,161,oppo,Oppo Find N2 5G,94990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor",Snapdragon 8+ Gen1,snapdragon,Octa Core,3.20,"12 GB RAM, 256 GB inbuilt",4520 mAh Battery with 67W Fast Charging,4520.0,67,12.0,256.0,"7.1 inches, 1792 x 1920 px, 120 Hz Display wit...",7.10,120,1792 x 1920,50 MP + 48 MP + 32 MP Triple Rear & 32 MP + 32...,3,2,Memory Card Not Supported,Memory Card Not Supported,50,32,0
306,308,samsung,Samsung Galaxy Z Flip 3,69999,84.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 888, Octa Core, 2.84 GHz Processor",Snapdragon 888,snapdragon,Octa Core,2.84,"8 GB RAM, 128 GB inbuilt",3300 mAh Battery with 15W Fast Charging,3300.0,15,8.0,128.0,"6.7 inches, 1080 x 2640 px, 120 Hz Display wit...",6.70,120,1080 x 2640,12 MP + 12 MP Dual Rear & 10 MP Front Camera,2,1,Memory Card Not Supported,Memory Card Not Supported,12,10,0
322,324,royole,Royole FlexPai 2,109999,87.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,"Snapdragon 865, Octa Core, 2.84 GHz Processor",Snapdragon 865,snapdragon,Octa Core,2.84,"8 GB RAM, 128 GB inbuilt",4450 mAh Battery,4450.0,-1,8.0,128.0,"7.8 inches, 1440 x 1920 px Display",7.80,60,1440 x 1920,64 MP + 16 MP + 8 MP Triple Rear & 32 MP Front...,3,1,Memory Card Not Supported,"Memory Card Supported, upto 256 GB",64,32,0
365,367,oppo,OPPO Find N Flip,89990,88.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Dimensity 9000, Octa Core, 3.05 GHz Processor",Dimensity 9000,dimensity,Octa Core,3.05,"8 GB RAM, 128 GB inbuilt",4300 mAh Battery with 44W Fast Charging,4300.0,44,8.0,128.0,"6.8 inches, 1200 x 2400 px, 120 Hz Display wit...",6.80,120,1200 x 2400,50 MP + 8 MP Dual Rear & 32 MP Front Camera,2,1,Memory Card Not Supported,Memory Card Not Supported,50,32,0
392,394,oppo,OPPO Find N2 Flip,70990,88.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Dimensity 9000 Plus, Octa Core, 3.2 GHz Processor",Dimensity 9000 Plus,dimensity,Octa Core,3.20,"8 GB RAM, 256 GB inbuilt",4300 mAh Battery with 44W Fast Charging,4300.0,44,8.0,256.0,"6.8 inches, 1080 x 2520 px, 120 Hz Display wit...",6.80,120,1080 x 2520,50 MP + 8 MP Dual Rear & 32 MP Front Camera,2,1,Memory Card Not Supported,Memory Card Not Supported,50,32,0
504,506,samsung,Samsung Galaxy Z Flip 4 5G,89999,88.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor",Snapdragon 8+ Gen1,snapdragon,Octa Core,3.20,"8 GB RAM, 128 GB inbuilt",3700 mAh Battery with 25W Fast Charging,3700.0,25,8.0,128.0,"6.7 inches, 1080 x 2640 px, 120 Hz Display wit...",6.70,120,1080 x 2640,12 MP + 12 MP Dual Rear & 10 MP Front Camera,2,1,Memory Card Not Supported,Memory Card Not Supported,12,10,0
570,572,lg,LG Wing 5G,54999,89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,"Snapdragon 765G , Octa Core, 2.4 GHz Processor",Snapdragon 765G,snapdragon,Octa Core,2.40,"8 GB RAM, 128 GB inbuilt",4000 mAh Battery with Fast Charging,4000.0,0,8.0,128.0,"6.8 inches, 1080 x 2460 px Display",6.80,60,1080 x 2460,64 MP + 13 MP + 12 MP Triple Rear & 32 MP Fron...,3,1,Memory Card Not Supported,"Memory Card (Hybrid), upto 2 TB",64,32,0
613,615,oukitel,Oukitel WP21,22990,82.0,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi",False,False,False,"Helio G99, Octa Core, 2.2 GHz Processor",Helio G99,helio,Octa Core,2.20,"12 GB RAM, 256 GB inbuilt",9800 mAh Battery with 66W Fast Charging,9800.0,66,12.0,256.0,"6.78 inches, 1080 x 2400 px, 120 Hz Display",6.78,120,1080 x 2400,64 MP + 20 MP + 2 MP Triple Rear & Main Front ...,3,1,Memory Card Not Supported,Memory Card (Hybrid),64,

In [ ]:
df1.loc[temp_df.index, 'os'] = 'Not Specified'

In [ ]:
df1['os'].value_counts()

Android v12                           393
Android v11                           274
Android v13                            91
Android v10                            69
Android v9.0 (Pie)                     29
Android v10.0                          23
iOS v16                                15
iOS v15                                12
Android v8.1 (Oreo)                    10
Memory Card Not Supported               6
iOS v14                                 6
Android v8.0 (Oreo)                     4
Android v11.0                           4
iOS v13                                 4
iOS v15.0                               3
Android v6.0 (Marshmallow)              2
Memory Card (Hybrid), upto 256 GB       2
Memory Card (Hybrid), upto 2 TB         2
HarmonyOS v2.0                          2
Android v5.1.1 (Lollipop)               2
Harmony v2.0                            2
Memory Card Supported, upto 256 GB      2
EMUI v12                                2
Android v12.1                     

In [ ]:
def os_extractor(text):

  if 'Android' in text:
    return 'android'
  elif 'iOS' in text:
    return 'ios'
  elif 'Not Specified':
    return text
  elif 'Harmony' in text or 'Hongmeng' in text or 'EMUI' in text:
    return 'other'

In [ ]:
df1['os'] = df1['os'].apply(os_extractor)

In [ ]:
export_df = df1.drop(columns=['index','sim','processor','ram','battery','display','camera','card'])

## 3. Cleaning Operations

In [5]:
def adjust_index(df):
    return (
        df
        .reset_index()
        .assign(index=lambda df_: df_['index'] + 2)
    )

In [6]:
def shift_battery_to_display(df):

    temp_df = df[df['index'].isin(battery_rows)]
    shifted_values = temp_df.iloc[:, 7:].shift(1, axis=1)

    # Update df with shifted values for specified rows and columns
    df.loc[temp_df.index, df.columns[7:]] = shifted_values.values

    return df



In [7]:
def update_card_columns(df):
    # Step 1: Update 'camera' with 'card' where 'camera' doesn't contain "MP" (for rows in camera_rows)
    df.loc[
        df['index'].isin(camera_rows) & ~df['camera'].fillna("").astype(str).str.contains('MP'),
        'camera'
    ] = df.loc[
        df['index'].isin(camera_rows) & ~df['camera'].fillna("").astype(str).str.contains('MP'),
        'card'
    ]
    
    # Step 2: Update 'card' to "Memory Card Not Supported" where 'card' contains "MP"
    df.loc[
        df['card'].fillna("").astype(str).str.contains('MP'),
        'card'
    ] = 'Memory Card Not Supported'
    
    # Step 3: Update 'os' with values from 'card' where 'card' doesn't contain "Memory Card"
    df.loc[
        ~df['card'].fillna("").astype(str).str.contains('Memory Card'),
        'os'
    ] = df.loc[
        ~df['card'].fillna("").astype(str).str.contains('Memory Card'),
        'card'
    ]
    
    # Step 4: Set 'card' to "Memory Card Not Supported" for rows that do not contain "Memory Card"
    df.loc[
        ~df['card'].fillna("").astype(str).str.contains('Memory Card'),
        'card'
    ] = 'Memory Card Not Supported'
    
    return df

In [8]:
def shift_processor_info(df):
    temp_df = df[df['processor_name'].str.contains('Core')][['processor_name', 'num_cores', 'processor_speed']].shift(1, axis=1)

    if not temp_df.empty: 
        df.loc[temp_df.index, ['processor_name', 'num_cores', 'processor_speed']] = temp_df
    else:
        print("No processors containing 'Core' found. Skipping shift.")  

    return df  

In [9]:
def fast_charging_extractor(df):
    return df.pipe(
        lambda ser: ser.apply(
            lambda item: item[0] if isinstance(item, list) and len(item) == 1 else (0 if isinstance(item, list) else -1)
        )
    )

In [10]:
def camera_extractor(df):
    return df.pipe(
        lambda ser: ser.apply(
            lambda text: '4' if 'Quad' in text else
                        ('3' if 'Triple' in text else
                         ('2' if 'Dual' in text else
                          ('Missing' if 'Missing' in text else '1')))
        )
    )

In [11]:
def os_extractor(df):
    return df.assign(
        os=df['os'].apply(lambda text: 'android' if 'Android' in text else
                          'ios' if 'iOS' in text else
                          text if 'Not Specified' in text else
                          'other' if any(x in text for x in ['Harmony', 'Hongmeng', 'EMUI']) else
                          text)
    )

In [12]:
def clean_data(df):
    return (
        df.assign(
            price=lambda df_: (
                df_.price
                .str.replace('₹', '', regex=False)  
                .str.replace(',', '', regex=False)
                .astype('int')
            )
        )
        .pipe(adjust_index)
        .loc[lambda df_: df_['price'] >= 3400] 
        .drop([645,857,882,925])
        .drop(582)
        .drop([376,754])
        .pipe(shift_battery_to_display)
        .drop([155, 271])
        .pipe(update_card_columns)
        .assign(
            os=lambda df_: df_['os'].where(df_['os'] != 'Bluetooth', np.nan),
            brand_name=lambda df_: df_['model'].str.split(' ').str.get(0).str.lower(),
            has_5g=lambda df_: df_['sim'].str.contains('5G'),
            has_nfc=lambda df_: df_['sim'].str.contains('NFC'),
            has_ir_blaster=lambda df_: df_['sim'].str.contains('IR Blaster'),
            processor_name=lambda df_: df_['processor'].str.split(',').str.get(0).str.strip(),
            num_cores=lambda df_: df_['processor'].str.split(',').str.get(1),
            processor_speed=lambda df_: df_['processor'].str.split(',').str.get(2)
        )
        .pipe(shift_processor_info)  
        .assign(
            processor_brand=lambda df_: df_['processor_name'].str.split(' ').str.get(0).str.lower(),
            num_cores=lambda df_: df_['num_cores'].str.strip().str.replace('Octa Core Processor','Octa Core').str.replace('Hexa Core Processor','Hexa Core'),
            processor_speed=lambda df_: df_['processor_speed'].str.strip().str.split(' ').str.get(0).str.replace('\u2009',' ').str.split(' ').str.get(0).astype(float),
            ram_capacity=lambda df_: df_['ram'].str.strip().str.split(',').str.get(0).str.findall(r'\b(\d+)\b').str.get(0).astype(float),
            internal_memory=lambda df_: df_['ram'].str.strip().str.split(',').str.get(1).str.strip().str.findall(r'\b(\d+)\b').str.get(0).astype(float)
        )
        .drop([486,627])
        .pipe(lambda df_: df_.assign(internal_memory=df_['internal_memory'].replace(1, 1024)))
        .assign(
            battery_capacity=lambda df_: df_['battery'].str.strip().str.split('with').str.get(0).str.strip().str.findall(r'\b(\d+)\b').str.get(0).astype(float),
            fast_charging = lambda df_: (
                            df_['battery']
                            .str.strip()
                            .str.split('with')
                            .str.get(1)
                            .str.strip()
                            .str.findall(r'\d{2,3}')
                            .pipe(fast_charging_extractor)  
            ),
            screen_size = lambda df_: (df_['display'].str.strip().str.split(',').str.get(0).str.strip().str.split(' ').str.get(0).astype(float)),
            resolution = lambda df_: (df_['display'].str.strip().str.split(',').str.get(1).str.strip().str.split('px').str.get(0)),
            refresh_rate = lambda df_: (df_['display'].str.strip().str.split(',').str.get(2).str.strip().str.findall(r'\d{2,3}').str.get(0).apply(lambda x: 60 if pd.isna(x) else x).astype(int)),
            num_rear_cameras = lambda df_: (df_['camera'].str.strip().str.split('&').str.get(0).pipe(camera_extractor)),
            num_front_cameras = lambda df_: (df_['camera'].str.strip().str.split('&').str.get(1).str.strip().fillna('Missing').pipe(camera_extractor)),
            primary_camera_rear = lambda df_: (df_['camera'].str.split(' ').str.get(0).str.replace('\u2009',' ').str.split(' ').str.get(0)),
            primary_camera_front= lambda df_: (df_['camera'].str.split('&').str.get(1).str.strip().str.split(' ').str.get(0).str.replace('\u2009',' ').str.split(' ').str.get(0)),
            extended_memory = lambda df_: (df_['card'].apply(lambda x:'0' if 'Not' in x else x.split('upto')).str.get(-1).str.strip().str.replace('Memory Card Supported','Not Specified'))
        )
        .pipe(os_extractor)
        .drop(columns=['index','sim','processor','ram','battery','display','camera','card'])
        
        # .shape 
    )


In [16]:
df_clean_1 = clean_data(df)
df_clean_1 = df_clean_1.copy() 

df_clean_1.loc[856, 'processor_name'] = 'Mediatek MT6739'

df_clean_1.loc[[483], ['ram_capacity','internal_memory']] = [12.0, '512']
df_clean_1.loc[69, 'camera'] = '50 MP'
df_clean_1.loc[69, 'card'] = 'Not Specified'
df_clean_1.loc[69, 'os'] = 'Not Specified'

C:\Users\PRAMIT DE\AppData\Local\Temp\ipykernel_17328\3360052537.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '512' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_clean_1.loc[[483], ['ram_capacity','internal_memory']] = [12.0, '512']


In [17]:
df_clean_1.shape

(980, 26)

## 4.Export the cleaned data

In [18]:
export_data(dataframe = df_clean_1, dir_name="interim", name="smartphone_cleaned_v1")

Data successfully exported to: E:\Learnings\Learning_curves\Smartphone_spec_score_analysis\data\interim\smartphone_cleaned_v1.csv
